In [57]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

1. Use a very simple way to train a model 
2. Preprocess the dataset into a trainable format 
3. Use a simple model to train 
4. Submit and see the result 
5. Then improve 

In [58]:
train = pd.read_csv('../data/sensor/train.csv')
labels = pd.read_csv('../data/sensor/train_labels.csv')
test = pd.read_csv('../data/sensor/test.csv')
print(train.head())

   sequence  subject  step  sensor_00  sensor_01  sensor_02  sensor_03  \
0         0       47     0  -0.196291   0.112395        1.0   0.329204   
1         0       47     1  -0.447450   0.134454        1.0  -0.658407   
2         0       47     2   0.326893  -0.694328        1.0   0.330088   
3         0       47     3   0.523184   0.751050        1.0   0.976991   
4         0       47     4   0.272025   1.074580        1.0  -0.136283   

   sensor_04  sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
0  -1.004660  -0.131638  -0.127505   0.368702       -0.1  -0.963873   
1   0.162495   0.340314  -0.209472  -0.867176        0.2  -0.301301   
2   0.473678   1.280479  -0.094718   0.535878        1.4   1.002168   
3  -0.563287  -0.720269   0.793260   0.951145       -0.3  -0.995665   
4   0.398579   0.044877   0.560109  -0.541985       -0.9   1.055636   

   sensor_10  sensor_11  sensor_12  
0  -0.985069   0.531893   4.751492  
1   0.082733  -0.231481   0.454390  
2   0.449221  -0.

## data preprocessing 
1. Windows size == time steps 
2. Reshape 

In [59]:
from tsai.all import StandardScaler
features = train.columns.tolist()[3:]
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.transform(test[features])

In [60]:
Window = 60
print(train.head(5))
train = train.drop(["sequence", "subject", "step"], axis=1).to_numpy()
train = train.reshape(-1, Window, train.shape[-1])
train = np.transpose(train, (0, 2, 1))

y_train = labels['state'].to_numpy()

test = test.drop(["sequence", "subject", "step"], axis=1).to_numpy()
test = test.reshape(-1, Window, test.shape[-1])
test = np.transpose(test, (0, 2, 1))

   sequence  subject  step  sensor_00  sensor_01  sensor_02  sensor_03  \
0         0       47     0  -0.073994   0.025755   0.529941   0.084226   
1         0       47     1  -0.168462   0.030764   0.529941  -0.166807   
2         0       47     2   0.122789  -0.157416   0.529941   0.084451   
3         0       47     3   0.196619   0.170766   0.529941   0.248882   
4         0       47     4   0.102151   0.244225   0.529941  -0.034093   

   sensor_04  sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
0  -0.595617  -0.081710  -0.037993   0.113685  -0.022186  -0.372260   
1   0.097598   0.214962  -0.062496  -0.267356   0.044458  -0.116728   
2   0.282421   0.805957  -0.028192   0.165228   0.311034   0.385976   
3  -0.333470  -0.451728   0.237261   0.293261  -0.066615  -0.384521   
4   0.237816   0.029248   0.167563  -0.167094  -0.199903   0.406597   

   sensor_10  sensor_11  sensor_12  
0  -0.513943   0.118173   0.121766  
1   0.042977  -0.050247   0.011916  
2   0.234122  -0.

In [61]:
print(train.shape)
print(y_train.shape)

(25968, 13, 60)
(25968,)


## flat time sequence into features. Merge 12 sensor, each sensor 60 time steps into features. 

In [31]:
sensors = [col for col in train.columns if 'sensor_' in col]
train_pivoted0 = train.pivot(index=['sequence', 'subject'], columns='step', values=sensors)
train_pivoted0.head()

sensor_00                                                    \
step                    0         1         2         3         4         5    
sequence subject                                                               
0        47      -0.196291 -0.447450  0.326893  0.523184  0.272025 -1.073416   
1        66      -6.657651  1.633694  1.863215 -2.846213  0.593509  2.469861   
2        66      -1.741886  3.402628 -0.632148  0.208655 -0.800618 -2.274343   
3        542      0.264297 -1.808346  0.078825  1.037094 -0.189335 -1.302937   
4        437      0.567233 -0.823802  1.476816 -1.710201 -0.408810  0.556414   

                                                          ... sensor_12  \
step                    6         7         8         9   ...        50   
sequence subject                                          ...             
0        47       0.350850 -0.304482 -0.216383  0.303709  ... -1.670077   
1        66      -1.202473 -0.829212  1.091190 -0.334621  ... -0.338022   
2        66       2.009274 -0.563369 -0.050232  0.800618  ... -0.172634   
3        542      1.078825  2.257342 -2.283617 -2.759660  ...  0.145354   
4        437      0.853168 -0.406491 -0.009274 -0.021638  ...  0.001705   

                                                                              \
step                    51        52        53        54        55        56   
sequence subject                                                               
0        47      -5.306479  3.621057  4.965047 -3.615090  0.791134  5.887042   
1        66       0.234868  0.030691  0.037511 -0.120205  0.056692  0.062234   
2        66       0.091645 -0.028986 -0.202472  0.332481 -0.013214  0.245524   
3        542     -0.045610 -0.098039 -0.049446  0.055840 -0.022165  0.054135   
4        437     -0.050725 -0.025149  0.106138 -0.098465 -0.040068  0.135976   

                                                  
step                    57         58         59  
sequence subject                                  
0        47      -7.211424 -26.738704 -50.658994  
1        66      -0.013640   0.098039  -0.115090  
2        66      -0.091219  -0.306905   0.046036  
3        542     -0.185422   0.311168  -0.166240  
4        437      0.121910  -0.059676  -0.087809  

[5 rows x 780 columns]

## Set index 

In [55]:
train = train.set_index(["sequence", "subject", "step"])
train.shape

(1558080, 13)

In [56]:
train.head(120)

sensor_00  sensor_01  sensor_02  sensor_03  sensor_04  \
sequence subject step                                                          
0        47      0     -0.196291   0.112395   1.000000   0.329204  -1.004660   
                 1     -0.447450   0.134454   1.000000  -0.658407   0.162495   
                 2      0.326893  -0.694328   1.000000   0.330088   0.473678   
                 3      0.523184   0.751050   1.000000   0.976991  -0.563287   
                 4      0.272025   1.074580   1.000000  -0.136283   0.398579   
...                          ...        ...        ...        ...        ...   
1        66      55    -0.483771   0.413866  -1.332820  -0.407965  -0.221653   
                 56    -0.354714   0.284664  -1.332820  -0.902655  -0.483216   
                 57     1.129830   0.543067  -1.332820   1.981416  -0.234852   
                 58    -0.069552  -0.005252  -1.332820  -0.997345   0.817604   
                 59    -0.102782  -0.784664  -1.001153  -0.222124   0.599769   

                       sensor_05  sensor_06  sensor_07  sensor_08  sensor_09  \
sequence subject step                                                          
0        47      0     -0.131638  -0.127505   0.368702       -0.1  -0.963873   
                 1      0.340314  -0.209472  -0.867176        0.2  -0.301301   
                 2      1.280479  -0.094718   0.535878        1.4   1.002168   
                 3     -0.720269   0.793260   0.951145       -0.3  -0.995665   
                 4      0.044877   0.560109  -0.541985       -0.9   1.055636   
...                          ...        ...        ...        ...        ...   
1        66      55    -0.113687  -0.313297  -0.056489        0.5  -0.982659   
                 56    -4.201945  -0.454463  -0.571756        1.3  -0.637283   
                 57     3.727749   1.637523   0.938168       -0.8   0.876445   
                 58     0.318624  -0.308743  -1.105344       -0.5   0.846098   
                 59     0.029170  -0.366120   0.051145        0.3  -0.319364   

                       sensor_10  sensor_11  sensor_12  
sequence subject step                                   
0        47      0     -0.985069   0.531893   4.751492  
                 1      0.082733  -0.231481   0.454390  
                 2      0.449221  -0.586420  -4.736147  
                 3     -0.434290   1.344650   0.429241  
                 4      0.812631   0.123457  -0.223359  
...                          ...        ...        ...  
1        66      55    -0.069806   0.135802   0.056692  
                 56     0.193876  -0.283951   0.062234  
                 57     0.366401   0.983539  -0.013640  
                 58     0.425227  -0.421811   0.098039  
                 59    -0.030967  -0.764403  -0.115090  

[120 rows x 13 columns]

In [ ]:
# Define LSTM model
from inception_time import InceptionTime

class LSTMModel(nn.Module):
    def __init__(self,  seq=60, input_size=13, lstm_dim=512, num_layers=2, num_classes=1,):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=lstm_dim, num_layers=num_layers, batch_first=True)
        self.lstm1 = nn.LSTM(input_size=lstm_dim, hidden_size=lstm_dim, num_layers=num_layers, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=lstm_dim, hidden_size=lstm_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(lstm_dim * seq, num_classes),
        )
    
    def forward(self, x):
        features , _ = self.lstm(x)
        # features , _ = self.lstm1(features)
        features , _ = self.lstm2(features)
        features = features.contiguous().view(features.size(0), -1)
        pred = self.fc(features)

        return pred

# Hyperparameters
# input_size = 13
# hidden_size = 1024
# num_layers = 2
# num_classes = 1
num_epochs = 10
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
# model = LSTMModel().to(device)
model = InceptionTime(in_channels=13, out_channels=32, num_classes=1, depth=6).to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.float().to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs = inputs.to(device)
            labels = labels.float().to(device)
            outputs = model(inputs)
            predicted = torch.round(torch.sigmoid(outputs)).squeeze()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f'Validation Accuracy: {100 * correct / total:.2f}%')